In [306]:
import pandas as pd

In [318]:
class CsvOperations:
    def transform_csv_to_dataframe(self, filename):
        return pd.read_csv(filename)
    
    def rename_column_in_place(self, df, original, new):
        df.rename(columns={original: new}, inplace=True)
        
    def transform_datataframe_to_csv(self, df, filename):
        df.to_csv(filename, index=False)
        
    def set_index_in_dataframe(self, df, column):
        df.set_index(column)
        
    

# Read the CSV's


In [319]:
operations = CsvOperations()
bills = operations.transform_csv_to_dataframe('bills.csv')
legislators = operations.transform_csv_to_dataframe('legislators.csv')
vote_results = operations.transform_csv_to_dataframe('vote_results.csv')
vote = operations.transform_csv_to_dataframe('votes.csv')


# Transforming some columns names 

In [320]:
operations.rename_column_in_place(bills, 'id', 'bill_id')
operations.rename_column_in_place(legislators, 'id', 'legislator_id')
operations.rename_column_in_place(vote_results, 'id', 'vote_result_id')
operations.rename_column_in_place(vote, 'id', 'vote_id')

# Set Indexes to better the speed of merge operations

In [321]:
operations.set_index_in_dataframe(bills, 'bill_id')
operations.set_index_in_dataframe(legislators, 'legislator_id')
operations.set_index_in_dataframe(vote_results, 'vote_result_id')
operations.set_index_in_dataframe(vote_results, 'vote_id')


# Merge all databases 

In [322]:
final_merge = (legislators.merge(vote_results, on='legislator_id')
                 .merge(vote, on='vote_id')
                 .merge(bills, on='bill_id'))

# Question 1

In [328]:

count = 0
supported_function = lambda row: count + 1 if row['vote_type'] == 1 else count
opposed_function = lambda row: count + 1 if row['vote_type'] == 2 else count

final_merge['num_supported_bills'] = final_merge.apply(supported_function, axis=1)
final_merge['num_opposed_bills'] = final_merge.apply(opposed_function, axis=1)
legislators_support_oppose = final_merge.groupby(['legislator_id', 'name']).agg({'num_supported_bills': 'sum', 'num_opposed_bills': 'sum'}).reset_index()
operations.transform_datataframe_to_csv(legislators_support_oppose, 'legislators-support-oppose-count.csv')

# Questão 2

In [329]:

bills_count = final_merge.groupby(['bill_id', 'sponsor_id']).agg({'num_supported_bills': 'sum', 'num_opposed_bills': 'sum'}).reset_index()
unique_values = final_merge['sponsor_id'].unique()
mask = final_merge['legislator_id'].isin(unique_values)
values = final_merge.loc[mask,'name']

for id_value in unique_values:
    bills_count.loc[bills_count['sponsor_id'] == id_value, 'name'] = values.values[0] if not values.empty else 'Unknown'
bills_count
operations.transform_datataframe_to_csv(bills_count, 'bills_result.csv')

,bill_id,sponsor_id,num_supported_bills,num_opposed_bills,name
0,2900994,400100,13,6,Unknown
1,2952375,412211,6,13,Unknown
